In [9]:
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
import copy
sc.stop()
sc = SparkContext('local')
spark = SparkSession(sc)

In [10]:
import os,glob
path0 = 'tsvfiles/'
df = spark.read.csv(
    os.path.join(path0,"SUNM.tsv"), sep=r'\t', header=False, mode="DROPMALFORMED")

dataColl=df.collect()
for row in dataColl[0:5]:
    print(row[0] + "," +str(row[1]))

S0558,well i vote out and then when you you know when you've do it and er
S0517,do vote out
S0558,yeah
S0517,yeah
S0558,and i think have this feel you know on occasion i've have these feel and i think oh i dunno if i've do the right thing


In [11]:
import xmltodict
def extract_time(f):
    with open(f) as fd:
        doc = xmltodict.parse(fd.read())
    s_time = doc['text']['header']['rec_length']
    s_seconds = (int(s_time[0])*3600)+ (int(s_time[2])*10+int(s_time[3]))*60 + int(s_time[5])*10+int(s_time[6])
    return s_seconds

s = extract_time('S376.xml')
print(s)

2026


In [12]:
def extract_speakers_n(df1, n):
    dataC = df1.collect()
    speaker = [dataC[0][0]]
    for row in dataC:
        if row[0] not in speaker:
            speaker.append(row[0])
        if len(speaker) == n:
            break
    #rdd1 = rdd1.rdd
    return df1, speaker

In [13]:
def extract_dialogues_dic_n(rdd1, speaker):
    n = len(speaker)
    dataC = rdd1.collect()
    speaker_dic = [{} for x in range(0,n)]
    
    
    for i in range(0,len(dataC)):
        if dataC[i][0] in speaker:
            p = speaker.index(dataC[i][0])
            if dataC[i][1]!=None:
                speaker_dic[p][i] = dataC[i][1].split()
    
    return speaker, speaker_dic

In [14]:
def get_vo(speaker, speaker_dic):
    n = len(speaker)
    voand = []
    voor = []
    speaker_vo = [[] for x in range(0,n)]
    vo_group = {}
    token_n = 0
    
    for i in range(0,n):
        for k,tokens in speaker_dic[i].items():
            
            for token in tokens:
                token_n = token_n+1
                if token not in voor:
                    voor.append(token)
                if token not in speaker_vo[i]:
                    speaker_vo[i].append(token)
    
    countand = 0
    if n==2:
        for token0 in speaker_vo[0]:
            if token0 in speaker_vo[1]:
                countand = countand+1
    if n==3:
        for token0 in speaker_vo[0]:
            if (token0 in speaker_vo[1]) and (token0 in speaker_vo[2]):
                countand = countand+1
    if n==4:
        for token0 in speaker_vo[0]:
            if (token0 in speaker_vo[1]) and (token0 in speaker_vo[2]) and (token0 in speaker_vo[3]):
                countand = countand+1
    countor = len(voor)
    overlap = countand/countor
    vo_group = {"speaker_number": n, "vo_overlap": countand, "vo_total": countor, "total_token_number":token_n, "overlap": overlap} 
    return vo_group
            

In [15]:
def get_vo_pairwise(speaker, speaker_dic):
    n = len(speaker)
    voand = []
    voor = []
    speaker_vo = [[] for x in range(0,n)]
    vo_group = {}
    token_n = 0
    
    for i in range(0,n):
        for k,tokens in speaker_dic[i].items():
            for token in tokens:
                token_n = token_n+1
                if token not in voor:
                    voor.append(token)
                if token not in speaker_vo[i]:
                    speaker_vo[i].append(token)
    
    if n==3:
        countand_list = [[] for j in range(0,3)]
        x = 0
        for s_i in range(0,n-1):
            for s_j in range(s_i+1,n):
                for token in speaker_vo[s_i]:
                    if token in speaker_vo[s_j]:
                        countand_list[x].append(token)
                x = x+1
        countand = 0               
        for c in range(0,3):
            countand = countand + len(countand_list[c])
        
        countor = len(voor)
        overlap = countand/(3*countor)
        
        vo_group = {"speaker_number": n, "vo_overlap": countand, "vo_total": countor, "total_token_number":token_n, "overlap": overlap,\
                    "s1_and_s2": len(countand_list[0]), "s1_and_s3": len(countand_list[1]),"s2_and_s3": len(countand_list[2])}
            
            
    if n==4:
        countand_list = [[] for i in range(0,6)]
        x = 0
        for s_i in range(0,n-1):
            for s_j in range(s_i+1,n):
                for token in speaker_vo[s_i]:
                    if token in speaker_vo[s_j]:
                        countand_list[x].append(token)
                x = x+1
        countand = 0               
        for c in range(0,6):
            countand = countand + len(countand_list[c])
        countor = len(voor)
        overlap = countand/(6*countor)
        
        vo_group = {"speaker_number": n, "vo_overlap": countand, "vo_total": countor, "total_token_number":token_n, "overlap": overlap,\
                    "s1_and_s2": len(countand_list[0]), "s1_and_s3": len(countand_list[1]),"s1_and_s4": len(countand_list[2]),\
                   "s2_and_s3": len(countand_list[3]),"s2_and_s4": len(countand_list[4]),"s3_and_s4": len(countand_list[5])}
    
    
    return vo_group

In [16]:
def get_vo_average_speaker(speaker, speaker_dic):
    n = len(speaker)
    voand = []
    voor = []
    speaker_vo = [[] for x in range(0,n)]
    vo_group = {}
    token_n = 0
    
    for i in range(0,n):
        for k,tokens in speaker_dic[i].items():
            token_n = token_n + len(tokens)
            for token in tokens:
#                 token_n = token_n+1
                if token not in voor:
                    voor.append(token)
                if token not in speaker_vo[i]:
                    speaker_vo[i].append(token)
                    
    if n==2:
        voand_2 = []
        for s0 in speaker_vo[0]:
            if s0 in speaker_vo[1]:
                if s0 not in voand_2:
                    voand_2.append(s0)
                    
        countand = len(voand_2)
        countor = len(voor)
        overlap = countand/countor
        
        vo_group = {"speaker_number": n, "vo_overlap": countand, "vo_total": countor, "total_token_number":token_n, "overlap": overlap}
    
    if n==3:
        countand_list = [[] for j in range(0,3)]
        x = 0
        for s_i in range(0,n):
            for s_j in range(0,n):
                if s_j != s_i:
                    for token in speaker_vo[s_i]:
                        if token in speaker_vo[s_j]:
                            if token not in countand_list[x]:
                                countand_list[x].append(token)
            x = x+1
            
        countand = 0               
        for c in range(0,3):
            countand = countand + len(countand_list[c])
        
        countor = len(voor)
        overlap = countand/(3*countor)
        
        vo_group = {"speaker_number": n, "vo_overlap": countand, "vo_total": countor, "total_token_number":token_n, "overlap": overlap,\
                    "s1_and_other": len(countand_list[0]), "s2_and_other": len(countand_list[1]),"s3_and_other": len(countand_list[2])}
            
            
    if n==4:
        countand_list = [[] for i in range(0,4)]
        x = 0
        
        for s_i in range(0,n):
            for s_j in range(0,n):
                if s_j != s_i:
                    for token in speaker_vo[s_i]:
                        if token in speaker_vo[s_j]:
                            if token not in countand_list[x]:
                                countand_list[x].append(token)
            x = x+1
            
        countand = 0               
        for c in range(0,4):
            countand = countand + len(countand_list[c])
        
        countor = len(voor)
        overlap = countand/(4*countor)

        
        vo_group = {"speaker_number": n, "vo_overlap": countand, "vo_total": countor, "total_token_number":token_n, "overlap": overlap,\
                    "s1_and_other": len(countand_list[0]), "s2_and_other": len(countand_list[1]),"s3_and_other": len(countand_list[2]),\
                   "s4_and_other": len(countand_list[3])}
    
    
    return vo_group

In [33]:
def get_vo_average_speaker_in_window(speaker, speaker_dic, start, end):
    n = len(speaker)
    voand = []
    voor = []
    speaker_vo = [[] for x in range(0,n)]
    vo_group = {}
    token_n = 0
    
    for i in range(0,n):
        for k,tokens in speaker_dic[i].items():
            if k>=start and k<end:
                token_n = token_n + len(tokens)
                for token in tokens:
#                 token_n = token_n+1
                    if token not in voor:
                        voor.append(token)
                    if token not in speaker_vo[i]:
                        speaker_vo[i].append(token)
    if n==2:
        voand_2 = []
        for s0 in speaker_vo[0]:
            if s0 in speaker_vo[1]:
                if s0 not in voand_2:
                    voand_2.append(s0)
                    
        countand = len(voand_2)
        countor = len(voor)
        overlap = countand/countor
        
        vo_group = {"speaker_number": n, "vo_overlap": countand, "vo_total": countor, "total_token_number":token_n, "overlap": overlap, "begin":start, "end":end}
    
    
    if n==3:
        countand_list = [[] for j in range(0,3)]
        x = 0
        for s_i in range(0,n):
            for s_j in range(0,n):
                if s_j != s_i:
                    for token in speaker_vo[s_i]:
                        if token in speaker_vo[s_j]:
                            if token not in countand_list[x]:
                                countand_list[x].append(token)
            x = x+1
            
        countand = 0               
        for c in range(0,3):
            countand = countand + len(countand_list[c])
        
        countor = len(voor)
        overlap = countand/(3*countor)
        
        vo_group = {"speaker_number": n, "vo_overlap": countand, "vo_total": countor, "total_token_number":token_n, "overlap": overlap,\
                    "s1_and_other": len(countand_list[0]), "s2_and_other": len(countand_list[1]),"s3_and_other": len(countand_list[2]),\
                   "begin":start, "end":end}
            
            
    if n==4:
        countand_list = [[] for i in range(0,4)]
        x = 0
        
        for s_i in range(0,n):
            for s_j in range(0,n):
                if s_j != s_i:
                    for token in speaker_vo[s_i]:
                        if token in speaker_vo[s_j]:
                            if token not in countand_list[x]:
                                countand_list[x].append(token)
            x = x+1
            
        countand = 0               
        for c in range(0,4):
            countand = countand + len(countand_list[c])
        
        countor = len(voor)
        overlap = countand/(4*countor)

        
        vo_group = {"speaker_number": n, "vo_overlap": countand, "vo_total": countor, "total_token_number":token_n, "overlap": overlap,\
                    "s1_and_other": len(countand_list[0]), "s2_and_other": len(countand_list[1]),"s3_and_other": len(countand_list[2]),\
                   "s4_and_other": len(countand_list[3]), 'begin':start, 'end':end}
    
    
    return vo_group
    
    

In [18]:
def number_of_slot_n(speaker_dic,length_u):
    number_of_u = 0
    for s_d in speaker_dic:
        number_of_u = number_of_u + len(s_d)
    
    number_slot = int(number_of_u - number_of_u%length_u)/length_u
    return int(number_slot)

In [19]:
def number_of_total_u(speaker_dic):
    number_of_u = 0
    for s_d in speaker_dic:
        number_of_u = number_of_u + len(s_d)
    
    return number_of_u

In [20]:
def get_vo_average_speaker_temporal_n(speaker, speaker_dic, length):
    slot_number = number_of_slot_n(speaker_dic, length)
    vo_dic_list = []
    total_u = 0
    
    for s in speaker_dic:
        total_u = total_u + len(s)
    
    for i in range(0,slot_number):
        vo_dic = get_vo_average_speaker_in_window(speaker,speaker_dic, 0, length*(i+1))
        vo_dic_list.append(vo_dic)
    
    vo_dic_end = get_vo_average_speaker_in_window(speaker, speaker_dic, 0, total_u)
    vo_dic_list.append(vo_dic_end)
    
    return vo_dic_list

In [21]:
out1 = extract_speakers_n(df,2)

In [22]:
out2 = extract_dialogues_dic_n(out1[0],out1[1])

In [26]:
out3 = get_vo(out2[0], out2[1])
print(out3)

{'speaker_number': 2, 'vo_overlap': 802, 'vo_total': 1857, 'total_token_number': 30732, 'overlap': 0.43187937533656434}


In [27]:
out4 = get_vo_pairwise(out2[0], out2[1])
print(out4)

{}


In [28]:
out5 = get_vo_average_speaker(out2[0], out2[1])
print(out5)

{'speaker_number': 2, 'vo_overlap': 802, 'vo_total': 1857, 'total_token_number': 30732, 'overlap': 0.43187937533656434}


In [29]:
out6 = get_vo_average_speaker_temporal_n(out2[0], out2[1], 50)
for o in out6:
    print(o['overlap'])

0.23225806451612904
0.35390946502057613
0.36423841059602646
0.38563829787234044
0.3886255924170616
0.3883089770354906
0.40264650283553877
0.3904923599320883
0.3885350318471338
0.3929663608562691
0.3962536023054755
0.39030955585464333
0.40235910878112713
0.4073139974779319
0.40931372549019607
0.3980370774263904
0.3896761133603239
0.38747553816046965
0.3881453154875717
0.38992537313432835
0.3893884892086331
0.39664310954063603
0.3943298969072165
0.3917438921651222
0.3933609958506224
0.39416058394160586
0.4017501988862371
0.40644654088050314
0.41245136186770426
0.40813253012048195
0.41263940520446096
0.41051862673484296
0.40883190883190884
0.41014799154334036
0.41562064156206413
0.4168391994478951
0.4165526675786594
0.4138162307176392
0.4128862590401052
0.4150453955901427
0.4151185137732223
0.4168781725888325
0.416141235813367
0.41594022415940224
0.4155363748458693
0.4169221336603311
0.4208611279563372
0.42254370102471367
0.4221556886227545
0.41927853341218213
0.4191090269636577
0.4186991

In [34]:
import pandas as pd
dl = []
temporal_vo = []
import glob,os
path0 = 'tsvfiles/'
path = 'vo_calculate/vo_group2_temporal_average_speaker'
files = glob.glob(os.path.join(path0, "S***.tsv"))
print(len(files))
for f in files:
    dff = spark.read.csv(f, sep=r'\t', header=False, mode="DROPMALFORMED")
    speaker_f = extract_speakers_n(dff,2)
    dialogue_dic_f = extract_dialogues_dic_n(speaker_f[0],speaker_f[1])
    df_vo_temporal = get_vo_average_speaker_temporal_n(dialogue_dic_f[0],dialogue_dic_f[1],50)
    for d in df_vo_temporal:
        d['file'] = f[len(f)-8:len(f)]
    temporal_vo = pd.DataFrame(df_vo_temporal)
    temporal_vo.to_csv(os.path.join(path, f[len(f)-8:len(f)-4]+'.csv'), index = False)

622


In [30]:
import pandas as pd
dl = []
import glob,os
path0 = 'tsvfiles/'
path = 'vo_calculate/'
files = glob.glob(os.path.join(path0, "****.tsv"))
print(files)
for f in files:
    dff = spark.read.csv(f, sep=r'\t', header=False, mode="DROPMALFORMED")
    speaker_f = extract_speakers_n(dff,2)
    dialogue_dic_f = extract_dialogues_dic_n(speaker_f[0],speaker_f[1])
    df_vo = get_vo_average_speaker(dialogue_dic_f[0],dialogue_dic_f[1])
    df_vo['file'] = f[len(f)-8:len(f)]
    dl.append(df_vo)

dl_s = pd.DataFrame(dl)
dl_s.to_csv(os.path.join(path,'vo_group_average_speaker_2.csv'), index = False)


['tsvfiles/SGN8.tsv', 'tsvfiles/S72E.tsv', 'tsvfiles/SUPV.tsv', 'tsvfiles/SYEZ.tsv', 'tsvfiles/SRW5.tsv', 'tsvfiles/SG4J.tsv', 'tsvfiles/S2AJ.tsv', 'tsvfiles/S24A.tsv', 'tsvfiles/SL3R.tsv', 'tsvfiles/S5HT.tsv', 'tsvfiles/SZAP.tsv', 'tsvfiles/S4C2.tsv', 'tsvfiles/SXVL.tsv', 'tsvfiles/SJJK.tsv', 'tsvfiles/SXWH.tsv', 'tsvfiles/SDGT.tsv', 'tsvfiles/SZYV.tsv', 'tsvfiles/STUM.tsv', 'tsvfiles/SVCT.tsv', 'tsvfiles/SVBG.tsv', 'tsvfiles/SHJH.tsv', 'tsvfiles/SV4W.tsv', 'tsvfiles/STV6.tsv', 'tsvfiles/SZME.tsv', 'tsvfiles/S32W.tsv', 'tsvfiles/SJE3.tsv', 'tsvfiles/SPXX.tsv', 'tsvfiles/S7QU.tsv', 'tsvfiles/SEM7.tsv', 'tsvfiles/S5ER.tsv', 'tsvfiles/SP5U.tsv', 'tsvfiles/S7FK.tsv', 'tsvfiles/SYDJ.tsv', 'tsvfiles/SUPB.tsv', 'tsvfiles/SRVD.tsv', 'tsvfiles/SN26.tsv', 'tsvfiles/S72Q.tsv', 'tsvfiles/SBS5.tsv', 'tsvfiles/S7P2.tsv', 'tsvfiles/SFPK.tsv', 'tsvfiles/SAZ8.tsv', 'tsvfiles/SAXQ.tsv', 'tsvfiles/SZXQ.tsv', 'tsvfiles/SXAQ.tsv', 'tsvfiles/S87R.tsv', 'tsvfiles/SHJJ.tsv', 'tsvfiles/SVCW.tsv', 'tsvfiles/S6